In [21]:
#!pip install -U selenium
#!pip install webdriver-manager

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

import time

In [22]:
options = webdriver.ChromeOptions()
options.add_argument('headless') # O mais importante, não renderiza parte gráfica
options.add_argument('window-size=1920x1080')

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
def input(numeroSorteio):
    for i in range(2):
        time.sleep(1)
        inputElement = driver.find_element(By.ID,"buscaConcurso")
        inputElement.clear()
        inputElement.send_keys(numeroSorteio)
        inputElement.send_keys(Keys.ENTER)

def getLotomania(numeroSorteio, tipo):

    if tipo == "quina":
        url = 'https://loterias.caixa.gov.br/Paginas/Quina.aspx'
    elif tipo == "megasena":
        url = 'https://loterias.caixa.gov.br/Paginas/Mega-sena.aspx'
    elif tipo == "lotofacil":
        url = 'https://loterias.caixa.gov.br/Paginas/Lotofacil.aspx'
    elif tipo == "lotomania":
        url = 'https://loterias.caixa.gov.br/Paginas/Lotomania.aspx'
    driver.get(url)
    input(numeroSorteio)
    

    

    resultado = driver.find_element(By.XPATH, "//div[@class='related-box gray-text no-margin']")
    
    acertos = driver.find_elements(By.XPATH, "//div[@class='related-box gray-text no-margin']/p/strong")

    separados = driver.find_elements(By.XPATH,'//div[@class="related-box gray-text no-margin"]/p/span')
    listaVencedores = []


    for i in range(len(separados)):
        print(separados[i].text)

    for i in range(len(acertos)):
        print(acertos[i].text)

    
    for i in range(len(separados)):
        if separados[i].text != '':
            listaVencedores.append(separados[i].text)


    dicionario = {}
    for i in range(len(listaVencedores)):
        dicionario[acertos[i].text] = listaVencedores[i]
    return dicionario

def getQuina(numeroSorteio, tipo):

        if tipo == "quina":
            url = 'https://loterias.caixa.gov.br/Paginas/Quina.aspx'
        elif tipo == "megasena":
            url = 'https://loterias.caixa.gov.br/Paginas/Mega-sena.aspx'
        elif tipo == "lotofacil":
            url = 'https://loterias.caixa.gov.br/Paginas/Lotofacil.aspx'
        elif tipo == "lotomania":
            url = 'https://loterias.caixa.gov.br/Paginas/Lotomania.aspx'
        driver.get(url)
        input(numeroSorteio)

        listaNumeros = []

        if tipo == "quina" or tipo == "megasena":
            html_list = driver.find_element(By.ID,'ulDezenas')
            items = html_list.find_elements(By.TAG_NAME,'li')
            for i in items:
                listaNumeros.append(i.text)
            print('listaNumeros: ', listaNumeros)    
        else:  
            items = driver.find_elements(By.XPATH,"//ul[@class='simple-container lista-dezenas " + tipo + "']/li")
            for i in items:
                listaNumeros.append(i.text)

        resultado = driver.find_element(By.XPATH, "//div[@class='related-box gray-text no-margin']")
        if tipo == "lotofacil":
            acertos = resultado.find_elements(By.XPATH, "//div[@class='related-box gray-text no-margin']/p/strong")
        else:
            acertos = resultado.find_elements(By.TAG_NAME,'strong')

        separados = driver.find_elements(By.XPATH,'//div[@class="related-box gray-text no-margin"]/p/span')
        listaVencedores = []
        listaAcertos = []



        for i in range(len(separados)):
            print(separados[i].text)

        print('-------------------') 
        print('tamanho: ', len(acertos))  
        for i in range(len(acertos)):
            if acertos[i].text.split(' ')[0].isdigit():
                listaAcertos.append(acertos[i].text)

            print(acertos[i].text)


        for i in range(len(separados)):

            if separados[i].text != '':
                listaVencedores.append(separados[i].text)

        print(len(acertos))
        print(len(listaVencedores))
        dicionario = {}
        dicionario['tipo_sorteio'] = tipo
        dicionario['sorteio'] = sorteio
        dicionario['numeros'] = listaNumeros

        if tipo == "lotofacil":
            for i in range(len(listaAcertos)):
                dicionario[listaAcertos[i]] = listaVencedores[i]
        else:
            for i in range(len(listaVencedores)):
                dicionario[listaAcertos[i]] = listaVencedores[i]
        return dicionario
    
    


#while(i -)
#valorPraCada = valorPremio / nVencedores
#numeroVencedores = re.sub('[^0-9]', '', resultado)



sorteio = '1981'
quina = getQuina(sorteio, "lotofacil")



from pymongo import MongoClient
import pymongo

client = MongoClient('mongodb://localhost:27017/')
db = client.dbconcursos
collection = db.megasena

collection.insert_one(quina).inserted_id


driver.close()